In [15]:
import os
import zipfile

In [16]:
os.environ["KAGGLE_CONFIG_DIR"] = r"C:\Users\nrsaa\.kaggle"

Extraction of the Data:

In [13]:
zip_path = "./data/rockpaperscissors.zip"
extract_dir = "./data/rps_data"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
    print("✓ Downloaded and extracted into:", os.path.abspath(extract_dir))

FileNotFoundError: [Errno 2] No such file or directory: './data/rockpaperscissors.zip'